## Model

In [31]:
import torch
import numpy as np
import chess
import chess.pgn
import chess.engine

# --- FEN to Tensor Encoding (12 channels: piece type x color) ---
piece_to_plane = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

def fen_to_tensor(fen):
    board = chess.Board(fen)
    tensor = np.zeros((12, 8, 8), dtype=np.float32)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            row = 7 - (square // 8)  # flip vertically to match tensor layout
            col = square % 8
            tensor[piece_to_plane[piece.symbol()], row, col] = 1.0
    return torch.tensor(tensor, dtype=torch.float32)

# --- Simple Feedforward Model ---
class TwoMoveMateSolver(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()                       # 12x8x8 → 768
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.output = nn.Linear(512, 4 * 64)              # 4 softmax heads over 64 squares

    def forward(self, x):
        x = self.flatten(x)           # [B, 768]
        x = F.relu(self.fc1(x))       # [B, 1024]
        x = F.relu(self.fc2(x))       # [B, 512]
        x = self.output(x)            # [B, 256]
        x = x.view(-1, 4, 64)         # [B, 4 moves, 64-square choices]
        return x


## Dataset

In [32]:
import torch
import pandas as pd
import chess
from torch.utils.data import Dataset

class TwoMoveMateDataset(Dataset):
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        fen = row['fen']
        move1 = chess.Move.from_uci(row['move1'])
        move2 = chess.Move.from_uci(row['move2'])

        # Encode board state
        input_tensor = fen_to_tensor(fen)

        # Encode moves as 4 target indices: from1, to1, from2, to2
        label = torch.tensor([
            move1.from_square,
            move1.to_square,
            move2.from_square,
            move2.to_square
        ], dtype=torch.long)

        return input_tensor, label

## Train

In [33]:
# --- Hyperparameters
batch_size = 64
epochs = 10
lr = 1e-3

In [29]:
# --- Data
from torch.utils.data import DataLoader
dataset = TwoMoveMateDataset('mate_in_2_simple.csv')
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [30]:
dataset.df

,fen,move1,move2
0,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6,g1f2
1,5r1k/pp4pp/5p2/1BbQp1r1/6K1/7P/1PP3P1/3R3R w -...,g4h4,g2g3
2,2r5/pR5p/5p1k/4p3/4r3/B4nPP/PP3P2/1K2R3 w - - ...,e1e4,b1a1
3,1r4k1/p4ppp/2Q5/3pq3/8/P6P/2PR1PP1/Rr4K1 w - -...,a1b1,d2d1
4,1r6/5k2/2p1pNp1/p5Pp/1pQ1P2P/2P4R/KP3P2/3q4 w ...,c4c6,a2a3
...,...,...,...
290956,8/5Nkp/3p1qp1/1P1B4/4p3/1Q5P/5PPK/2r5 w - - 1 35,f2f3,g2g3
290957,6k1/4rpp1/2Bp3p/p7/P1P1Pp1q/2P1b3/2Q3rP/R1B4K ...,c2g2,g2f1
290958,4r2k/1NR2Q1p/4P1n1/pp1p4/3P4/4q3/PP4PP/6K1 w -...,g1h1,f7f1
290959,3r3k/p5pp/8/5R2/1BQ1p3/P3q3/Bb4PP/6K1 w - - 0 28,g1f1,b4e1


In [34]:
# --- Model and training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TwoMoveMateSolver().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_moves = 0
    total_moves = 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        output = model(inputs)  # shape: [B, 4, 64]

        loss = 0
        for i in range(4):
            loss += criterion(output[:, i, :], labels[:, i])
            preds = output[:, i, :].argmax(dim=1)
            correct_moves += (preds == labels[:, i]).sum().item()
            total_moves += labels.size(0)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    accuracy = correct_moves / (total_moves)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")


Epoch 1 | Loss: 7.8956 | Accuracy: 0.4663
Epoch 2 | Loss: 5.7035 | Accuracy: 0.5891


In [ ]:
torch.save(model.state_dict(), "twomove_model.pt")
print("Model saved as 'twomove_model.pt'")

## Convert to just Mate in 2 csv

In [27]:
# Load the Lichess puzzle CSV (decompressed)
# df = pd.read_csv("lichess_db_puzzle.csv", low_memory=False)

# # Step 1: Filter for mate-in-2 puzzles
# df = df[df["Themes"].str.contains("mateIn2", na=False)]

# # Step 2: Keep only puzzles where White moves first
# df = df[df["FEN"].str.split().str[1] == "w"]

# # Step 3: Get player's first and second moves (0 and 2)
# df["move_list"] = df["Moves"].str.split()
# df = df[df["move_list"].str.len() >= 3]
# df["move1"] = df["move_list"].apply(lambda m: m[0])
# df["move2"] = df["move_list"].apply(lambda m: m[2])

# # Step 4: Save just the FEN and two moves
# df = df[["FEN", "move1", "move2"]].rename(columns={"FEN": "fen"})
# df.to_csv("mate_in_2_simple.csv", index=False)